# Building a program to solve ISD-IFAM featuring:
- Ability to assign optional flights
- Calculation of minimum cost, spilled passengers and flight deletion 
- User-friendly interface to receive inputs and display outputs
- Ability to solve any generic fleet assignment problem 

# The objective of this model is to minimize total cost 
- Which flights should be canceled?
- Which fleet will make the available flights?
- How many passengers will be spilled then recaptured or lost? 
- How many aircraft will be sufficient to fulfill the given flights?
- How much does it cost the airline for that schedule ?


In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
from itertools import product
from mip import Model, xsum, maximize, minimize, BINARY, INTEGER
from flask import Flask, jsonify
from flask_restful import Api, Resource, reqparse
import pandas as pd
import json
from flask_cors import CORS
from flask import Flask, Blueprint
from flask_cors import CORS
from flask_restful import Resource, Api
import config

In [ ]:
airports_json = open('airports.json','r')
airports_data = airports_json.read()
Airports = json.loads(airports_data)

Orgin = []
Destination = []
Departure = []
Arrival = []
flight_index = []
options = ['mandatory','optional']
option = []
I_Orgin = []
I_Destination = []
I_option = []
iti_index = []
Demand = []
Fare_I = []
all_country = []
matt = []
matt1 = []
arr_optional_flight = []
arr_man_flight = []
p_od = []
AC_k = []
list_nodes = []
OpI = []
OpI_index = []
Demand_OpI = []
Fare_OI = []
cost = []
#######################################################################################################################

# Api
app = Flask(__name__)
CORS(app)
api = Api(app)
#parser = reqparse.RequestParser()

class FleetAssignment(Resource):
    #global size_K, i_orgin, i_destination, I_Opt_Mand, no_connecting_flights, demand, fare, K, cap, iti, list_iti, OC

    def get(self):
        return Airports

    def post(self):
        
        #global size_K
        global capacit, no_available_A, I_Orgin, I_Destination, Demand, Fare_I, I_option, cost, list_iti, option, Destination, Orgin, iti_index, no_available_AC, capacity, size_K, i_orgin, i_destination, I_Opt_Mand, no_connecting_flights, demand, fare, K, cap, iti, list_iti, OC

        parser = reqparse.RequestParser()

        
        parser.add_argument('i_orgin', type = str)                        # Orgin of itinerary
        parser.add_argument('i_destination', type = str)                  # Destination of itinerary
        #parser.add_argument('It', type = int)                              # No of itineraries  
        parser.add_argument('I_Opt_Mand', type = str)                     # ['mandatory','optional'] 
        parser.add_argument('no_connecting_flights', type = int)          # No of connecting flights  
        parser.add_argument('demand', type = int)                         # itinerary demand  
        parser.add_argument('fare', type = int)                           # itinerary fare
        parser.add_argument('K', type = int)                              # No of fleets
        parser.add_argument('OC', type = list)                             # Cost for each fleet in each flight
        parser.add_argument('cap', type = list)                            # Capacity of A/C fleet
        parser.add_argument('iti', type = list)                            # index of of flight no... in itinerary no...
        parser.add_argument('size_K', type = list)                         # No of available AC of Fleet
        
        
        args = parser.parse_args()
        print(args)
        i_orgin = args['i_orgin']
        i_destination = args['i_destination']
        #I = args['It']
        I_Opt_Mand = args['I_Opt_Mand']
        no_connecting_flights = args['no_connecting_flights']
        demand = args['demand']
        fare = args['fare']
        K = args['K']
        #OC = args['OC']
        cap = args['cap']
        iti = args['iti']
        size_K = args['size_K']
        
        if no_connecting_flights > 1:
            OC = []
        else:
            OC = args['OC']
        
        I_Orgin.append(i_orgin)
        I_Destination.append(i_destination)
        I_option.append(I_Opt_Mand)            
        Demand.append(demand)
        Fare_I.append(fare)
        no_available_A = size_K.copy()
        capacit = cap.copy()
        capacity = np.array(capacit)
        no_available_AC = np.array(no_available_A)
        #cost = OC.copy()
        iti_index.append(iti)
        
        for y in OC:
            if y != 0 :
                cost.append(y)
        print(OC,cost)
    
           

        if no_connecting_flights == 1: 
            list_iti = []
            Orgin.append(i_orgin)
            Destination.append(i_destination)  
            option.append(I_Opt_Mand)
            
            
        print(size_K)
            
#########################################################################################################

class Solution(Resource):
    def post(self):
        
        parser = reqparse.RequestParser()

        parser.add_argument('solvess', type = str)                              # No of itineraries  

        args = parser.parse_args()
        
        print(args)
        
        sol = args['solvess']
        #for j in range(len(iti_index)) 
        
        if sol == 'solve':
            I = len(I_Orgin)

            F = np.array(Orgin).size
            for f in range(F):
                flight_index.append(f+1)         
            #list_of all countries orgin or destination


            OPC = np.array(cost)
            OPC = np.reshape(OPC, (F, K))

            for o in Orgin:
                for d in Destination:
                    if o not in all_country:
                        all_country.append(o)
                    if d not in all_country:
                        all_country.append(d)
            #print(all_country)
            no_stations = np.size(all_country)

            #list_of 1 & -1 refer to orgin or destination
            for air in range(len(all_country)):
                       for f in range(F):
                        if all_country[air] == Orgin[f]:
                                matt.append(-1)
                        elif all_country[air] == Destination[f]:
                                matt.append(1)
                        else:
                                matt.append(0)
            mat=np.array(matt)
            mat = mat[mat != 0]

        #Nodes

            df = pd.DataFrame({'Flight Index':flight_index,'Orgin':Orgin,'Destination':Destination,'Category':option})
            df.set_index('Flight Index',inplace=True)        
        #df = pd.DataFrame({"Flight Index": flight_index ,"Orgin":Orgin ,"Destination":Destination ,"Category": option})
        #return({'Flight Index':flight_index,'Orgin':Orgin,'Destination':Destination,'Category':option})
            for air in range(len(all_country)):
                       for f in range (F):
                        if all_country[air] == Orgin[f]:
                                matt1.append(df.Orgin.index[f])
                        elif all_country[air] == Destination[f]:
                                matt1.append(df.Destination.index[f])
                        else:
                                matt1.append(0)
            mat1 = np.array(matt1)
            mat1 = mat1[mat1 != 0]        

            # Category of flights

            for f in range (F):
                if option[f] == options[0]:
                    arr_man_flight.append(df.Category.index[f])
                elif option[f] == options[1]:
                    arr_optional_flight.append(df.Category.index[f])

            arr_optional_flights = np.array(arr_optional_flight)
            arr_man_flights = np.array(arr_man_flight)
            no_man_flights = np.size(arr_man_flights)
            no_optional_flights = np.size(arr_optional_flights)
            no_optional_iti = I_option.count('optional')

            for i in range(I):
                p_od.append([f'{I_Orgin[i]},{I_Destination[i]}'])



                    #Input Fleets


            for e in range(K):
                AC_k.append(e+1)

            #No of nodes in each station

            for air in (all_country):
                list_nodes.append(Orgin.count(air)+Destination.count(air))

            #Input cost



            no_nodes = F * 2   

            ############################ Decision Variables ##########################
            X_var = K * F
            Y_var = K * no_nodes
            T_var = I * (I + 1)
            Z_var = no_optional_iti


    ############################# Coverage Constraints #########################
    # mandatory
            beq1 = np.ones((no_man_flights, 1), int)
            coeff_man_flt_const = np.zeros((no_man_flights, (X_var + Y_var + T_var + Z_var)), int)

            for x in range(0, no_man_flights):
                y = arr_man_flights[x]
                for z in range(0, K):
                    n = z + K * y - K
                    coeff_man_flt_const[x, n] = 1
            Aeq1 = coeff_man_flt_const

            # optional
            beq2 = np.ones((no_optional_flights, 1), int)
            coeff_optional_flt_const = np.zeros((no_optional_flights, (X_var + Y_var + T_var + Z_var)), int)
            for x in range(0, no_optional_flights):
                y = arr_optional_flights[x]
                for z in range(0, K):
                    n = z + K * y - K
                    coeff_optional_flt_const[x, n] = 1

            Aeq2 = coeff_optional_flt_const

         ########################## Balance #######################

            d1 = []
            sum = 0
            for n in range(np.size(list_nodes)):
                sum = sum + list_nodes[n]
                d1.append(sum)
            d = np.array(d1)
            #print(d)


            t = np.array([0])
            e = np.concatenate((t, d), axis=0)
            for i in range(np.size(e)):
                e[i] = e[i] + 1

            for i in range(np.size(d)):
                d[i] = d[i] + 1
            #print(d)


            column1 = np.zeros((np.size(d)), int)
            for n in range(0,np.size(d)):
                column1[n] = 1 + (d[n] - 1) * K
            #print(column1)


            column2 = np.zeros((np.size(e)), int)
            for n in range(0,np.size(e)):
                column2[n] = 1 + (e[n] - 1) * K
            #print(column2)

            g = []
            for i in range(np.size(list_nodes)):
                r = list_nodes[i] * K
                g.append(r)
            g_array=np.array(g)


            m1 = []
            sum = 0
            for n in range(np.size(g_array)):
                sum = sum + g_array[n]
                m1.append(sum)
            m = np.array(m1)

            # Y_variables
            nodes = np.diag(np.ones((Y_var), int))

            for i in range(0, Y_var - K):
               nodes[i, i + K] = -1

            for h in range(K):
                for i in column1:
                    if i > Y_var:
                        break                
                    nodes[i-1 - K , i-1] = 0
                    nodes[i - K -1 + h, i-1+h] = 0
            for h in range(K):
                for j in range(np.size(m)):
                    if column2[j] > Y_var:
                        break
                    if m[j] > Y_var:
                        break
                    nodes[m[j]-K,column2[j]-1] = -1
                    nodes[m[j] - K+h, column2[j]-1+h] = -1

            # X_variables
            coeff_X = np.zeros((Y_var, X_var), int)

            column = np.zeros((no_nodes), int)
            for n in range(0, no_nodes):
                column[n] = 1 + (mat1[n] - 1) * K

            for n in range(0,no_nodes):
                for i in range(K):
                    coeff_X[n*K+i,column[n]-1+i] = mat[n]


            coeff_T_Z = np.zeros((Y_var, (T_var + Z_var)), int)
            coeff_X_Y_T_Z = np.concatenate((coeff_X,nodes,coeff_T_Z), axis=1)

            beq3 = np.zeros((Y_var,1),int)
            Aeq3 = coeff_X_Y_T_Z

           ####################### Resource availability ########################

            # Y_variables
            coeff_Y_RAC = np.zeros((K, Y_var), int)
            y = np.zeros((no_stations),int)
            y[0] = 1

            for n in range(1,no_stations):
                y[n] = y[n-1] + list_nodes[n - 1]

            column = np.zeros((no_stations), int)
            for n in range(0, no_stations):
                column[n] = 1 + (y[n] - 1) * K

            beq4 = np.zeros((K, 1), int)

            for n in range(0, K):
                beq4[n] = no_available_AC[n]
                for k in range(0,no_stations):
                    coeff_Y_RAC[n,column[k]-1+n] = 1

            Aeq4 = coeff_Y_RAC
            coeff_X_RA = np.zeros((K,X_var),int)
            coeff_T_Z_RA = np.zeros((K,T_var + Z_var),int)
            Aeq4 = np.concatenate((coeff_X_RA,coeff_Y_RAC,coeff_T_Z_RA), axis=1)

            ################################## Sawsaw ####################################

            #LHS
            DemandDum = Demand.copy()
            DemandDum.append(0)

            r_od = p_od.copy()
            r_od.append(['Dummy'])
            tpr = np.zeros((len(p_od),len(r_od)),int)

            for i in range(len(p_od)):
                i1=p_od[i]
                d1=DemandDum[i]
                for ii in range(len(r_od)):
                    i2=r_od[ii]
                    d2=DemandDum[ii]
                    if (i1==i2 and d1>d2) or (ii == len(r_od) - 1):
                        tpr[i,ii]=1

            c=0
            tpr1m=np.zeros((I,I*(I+1)),int)
            for p in range(I):
    
                for r in range(I+1):
        
                    tpr1m[p][c]=tpr[p][r]
                    c+=1
            for opt in range(len(I_option)):
                 if I_option[opt] == 'optional':
                        OpI.append(p_od[opt])
                        OpI_index.append(opt)
                        Demand_OpI.append(Demand[opt])
                        Fare_OI.append(Fare_I[opt])

            Demand_Correction = np.ones((len(p_od),len(OpI)),int)*5            # Assuming the deltaqp to be 5

            for i in range(len(p_od)):
                for q in range(len(OpI)):
                    if p_od[i] == OpI[q] and i == OpI_index[q]:
                        Demand_Correction[i,q] = 0
                    elif p_od[i]==OpI[q]:
                        Demand_Correction[i,q] = -3


            Demand_Correction_transpose = Demand_Correction.T                    #obj fun

            Demand_Correction_sum = np.sum(Demand_Correction, axis=1)           # Sum of each row
            Demand_Correction_RHS = Demand_Correction_sum.reshape(len(Demand_Correction_sum),1)     # Transposing it


            #Transposing Demand
            Demandc = np.array(Demand)
            DemandX = Demandc.reshape(len(p_od),1)

            RHS_Demand = DemandX + Demand_Correction_RHS


            coefficient_X_Y_Demand = np.zeros((len(p_od), X_var + Y_var),int)
            coefficient_Z_Demand = Demand_Correction                                                      #np.zeros((len(p),len(OpI)),int)
            coeff_X_Y_T_Z_Demand = np.concatenate((coefficient_X_Y_Demand , tpr1m, coefficient_Z_Demand), axis=1)


            A_eq_Demand = coeff_X_Y_T_Z_Demand
            b_eq_Demand = RHS_Demand

            ################################ Capacity ####################################



            bpr=np.ones((I,I+1),int)*0.9     #tested

            ################# delta(flight-itinerary incident matrix) ##################
            delta=np.zeros((F,I), int)
            for no in range(I):
                ind=no
                flights=iti_index[ind]
                for f in flights:
                    delta[f-1, no] = 1

            #######################  tpr(IIM)   #########################

            trp=np.transpose(tpr)
            #print(trp) #tested

            ############  recaptured demand of flight f  ################
            Rdm1=np.zeros((F,I*(I+1)),float)
            for f in range(F):
                c=0
                for r in range(I + 1):
                    for p in range(I):
                      if r==I:
                          Rdm1[f][c]=0
                      else:
                          Rdm1[f][r + c]= delta[f][p] * bpr[p][r]* trp[p][r]
                          c+=1

            #############  spilled demand of flight f  ##############
            SDm1=np.zeros((F,I*(I+1)),float)
            for f in range(F):
                c2=0
                for p in range(I):
                    for r in range((I+1)):
                        SDm1[f][c2]=tpr[p][r]*delta[f][p]
                        c2+=1

            ################  seat of flight f ###############
            seat_m = np.zeros((F,F*K),int)
            c3 = 0
            for f in range(F):
                for e in range(K):
                    seat_m[f][e+c3] = capacity[e]
                c3+=2

            ############# RHS __Qf ###################
            Qfm=np.zeros((F,1),int)
            for f in range(F):
                sum_1=0
                for p in range(I):
                    Qf=Demand[p]*delta[f][p]
                    sum_1+=Qf
                Qfm[f, 0]=sum_1

            ###################### demand correction term ################        #tested
            dem_corr_m=np.zeros((F,Z_var),int)
            for f in range(F):
                for q in range(Z_var):
                   sum_on_p=0
                   for p in range(I):
                     dem_corr=delta[f][p]*Demand_Correction_transpose [q][p]
                     sum_on_p+=dem_corr
                   dem_corr_m[f][q]=sum_on_p
            dem_cor_RHS=np.sum(dem_corr_m,axis=1)
            dem_cor_RHS=np.transpose([dem_cor_RHS])

            ######  concatenation ###########
            diff_Dm=SDm1-Rdm1
            MatrixA_capacity= -1 * (np.concatenate((seat_m, np.zeros((F, Y_var), float), diff_Dm, dem_corr_m), axis=1))
            MatrixB_capacity= -1 * (Qfm + dem_cor_RHS)

            #print(np.shape(MatrixA_capacity))
            #print(np.shape(MatrixB_capacity))

            ################################# 1st ############################
            u=pd.DataFrame(delta)

            No_F_OIti=[] # Number of flights in each of the optional itineraries
            for i in OpI_index:
                No_F_OIti.append((u[i]).sum())
            Z_col=np.array(No_F_OIti)

            # To get total number of flights in optional itineraries (no.rows needed in this constraint)
            sum = 0
            for i in Z_col:
                sum = sum + i
            no_rows = sum

            # Get End, Step, Start
            end = []
            sum = 0
            for n in range(np.size(Z_col)):
                sum = sum + Z_col[n]
                end.append(sum)
            End = np.array(end)
            #
            step=[]
            for n in Z_col:
                step.append(n - 1)
            Step=np.array(step)
            #
            start=[]
            for n in range(np.size(End)):
                start.append(End[n] - Step[n])
            Start=np.array(start)
            # Coefficient of Z variables
            coefficient_Z=np.zeros((no_rows,Z_var),int)

            for i in range(np.size(Z_col)):
                coefficient_Z[np.arange(Start[i]-1,End[i],1),i]=1


            ############################################## X_Variables ############################################################
            array_x = []
            for i in OpI_index:
                array_x.append(iti_index[i])

            a=[]
            for i in array_x:
                for n in i:
                    a.append(n)
            a_array=np.array(a)

            column = np.zeros((np.size(a_array)), int)
            for n in range(0,np.size(a_array)):
                column[n] = 1 + (a_array[n] - 1) * K

            # Coefficient of X variables
            coefficient_X=np.zeros((no_rows,X_var),int)

            for i in range(np.size(column)):
                for n in range(K):
                    coefficient_X[i, column[i] - 1 + n]= -1

            ########################################## Coefficient of Y,T Variables ################################################################
            coefficient_Y_T=np.zeros((no_rows,Y_var + T_var),int)

            ############################################ Coefficient of first additional constraint ##############################################################
            coeff_X_Y_T_Z = np.concatenate((coefficient_X, coefficient_Y_T,coefficient_Z), axis=1)

            ############################################### Matrices of first additional constraint ###########################################################
            Aeq5 = coeff_X_Y_T_Z
            beq5 = np.zeros((no_rows,1),int)

            ####################################### 2nd ##################################
            ############################################ Z_Variables ##############################################################
            u=pd.DataFrame(delta)

            No_F_OIti=[]
            for i in OpI_index:
                No_F_OIti.append((u[i]).sum())
            Z_sum_col=np.array(No_F_OIti)

            #Right hand side
            ones=np.ones((no_optional_iti, 1), int)

            Nq = np.reshape(Z_sum_col, (np.size(Z_sum_col), 1))

            RHS=np.array((ones) - (Nq))

            # Coefficient_Z
            Coefficient_Z=np.diag(np.ones((no_optional_iti), int))

            ############################################## X_Variables ############################################################
            array_x = []
            for i in OpI_index:
                array_x.append(iti_index[i])

            a=[]
            for i in array_x:
                for n in i:
                    a.append(n)
            a_array=np.array(a)

            column = np.zeros((np.size(a_array)), int)
            for n in range(0,np.size(a_array)):
                column[n] = 1 + (a_array[n] - 1) * K

            # Get End, Step, Start
            end = []
            sum = 0
            for n in range(np.size(Z_sum_col)):
                sum = sum + Z_sum_col[n]
                end.append(sum)
            End = np.array(end)

            step=[]
            for n in Z_sum_col:
                step.append(n - 1)
            Step=np.array(step)

            start=[]
            for n in range(np.size(End)):
                start.append(End[n] - Step[n])
            Start=np.array(start)

            # Coefficient_X
            Coefficient_X=np.zeros((no_optional_iti, X_var), int)

            a=[]
            for i in range(no_optional_iti):
                a=(column[Start[i]-1:End[i]])
                for n in a:
                    for k in range(K):
                        Coefficient_X[i, a - 1 + k] = -1

            ########################################## Coefficient of Y,T Variables ################################################################
            coefficient_Y_T=np.zeros((no_optional_iti,Y_var + T_var),int)

            ############################################ Coefficient of second additional constraint ##############################################################
            coeff_X_Y_T_Z = np.concatenate((Coefficient_X * -1, coefficient_Y_T, Coefficient_Z * -1), axis=1)

            ############################################### Matrices of seco additional constraint ###########################################################
            beq6 = RHS * -1
            Aeq6 = coeff_X_Y_T_Z

            #print(np.shape(Aeq6))
            #print(np.shape(beq6))

            ###############################################################################################
             # Objectve Function Part

            fare_times_deltaDqp = Demand_Correction_transpose*Fare_I
            #print(fare_times_deltaDqp)

            fare_times_deltaDqp_sum_rows = np.sum(fare_times_deltaDqp, axis=1)
            #print(fare_times_deltaDqp_sum_rows)

            Demand_OpII= np.array(Demand_OpI)                # Make it array to be multiplicated
            fareO_times_DemandO = Demand_OpII * Fare_OI
            #print(fareO_times_DemandO)


            Zq_Coeff_ObjFun = -1*(fareO_times_DemandO - fare_times_deltaDqp_sum_rows)



                     #######CAPACITY obj fn ###########
            ####S=tpr*fare_p        #tested

            Fare_I.append(0)

            S=np.zeros((1,I*(I+1)),int)
            c4=0
            for p in range(I):
                for r in range(I+1):
                    S[0][c4]=tpr[p][r]*Fare_I[p]
                    c4+=1

             ###########M=tpr*fare_r*bpr############### #tested
            M = np.zeros((1, I * (I + 1)), float)
            c4 = 0
            for p in range(I):
                for r in range(I + 1):
                    M[0][c4] = float(tpr[p][r]) * Fare_I[r]*bpr[p][r]
                    c4+=1


            coeff_obj_T_1 = S - M
            coeff_obj_T_2 = coeff_obj_T_1[0]
            coeff_obj_X = np.array(cost)
            coeff_obj_Rest = np.zeros((Y_var),int)
            coeff_obj_Zq = Zq_Coeff_ObjFun



            ###############################################################################
            # TEST #
            coeff_obj = np.concatenate((coeff_obj_X, coeff_obj_Rest), axis=0)

            matrix_equ = np.concatenate((Aeq1, Aeq3), axis=0)

            matrix_inequ = np.concatenate((Aeq2, Aeq4,Aeq5,Aeq6,A_eq_Demand,MatrixA_capacity), axis=0)

            matrix_b_equ = np.concatenate((beq1,beq3), axis=None)

            matrix_b_inequ = np.concatenate((beq2,beq4,beq5,beq6,b_eq_Demand,MatrixB_capacity), axis=None)

            ###############################################################################

            flights = coeff_obj_X
            rest = coeff_obj_Rest
            TPR = coeff_obj_T_2
            ZQ = coeff_obj_Zq


            ################################################################################
            A = matrix_inequ
            b = matrix_b_inequ

            Aeq = matrix_equ
            beq = matrix_b_equ


            row,col = np.shape(A)
            rowq,colq = np.shape(Aeq)


            #################################################################################################################################################

            # Building The Model #

            m = Model()

            # Adding The Decision Variables and Identifying which is " BINARY " or " INTEGER " #

            x = [m.add_var(var_type=BINARY) for i in range(len(flights))]
            y = [m.add_var(var_type=INTEGER , lb=0) for i in range(len(rest))]
            t = [m.add_var(var_type=INTEGER , lb=0) for i in range(len(TPR))]
            z = [m.add_var(var_type=BINARY) for i in range(len(ZQ))]

            # Maximizing or Minimizing the Objective Function #

            m.objective = minimize(xsum(flights[i] * x[i] for i in range(len(flights))) + xsum(rest[j] * y[j] for j in range(len(rest))) + xsum(TPR[n] * t[n] for n in range(len(TPR))) + xsum(ZQ[q] * z[q] for q in range(len(ZQ))))

            # Adding The Constraints with Equality and Inequality signs #

            for r in range(rowq):
                m += xsum(Aeq[r][F] * x[F] for F in range(len(flights))) + xsum(Aeq[r][REST+len(flights)] * y[REST] for REST in range(len(rest))) + xsum(Aeq[r][T+len(flights)+len(rest)] * t[T] for T in range(len(TPR))) + xsum(Aeq[r][Z+len(flights)+len(rest)+len(TPR)] * z[Z] for Z in range(len(ZQ))) == beq[r]

            for r in range(row):
                m += xsum(A[r][F] * x[F] for F in range(len(flights))) + xsum(A[r][REST+len(flights)] * y[REST] for REST in range(len(rest))) + xsum(A[r][T+len(flights)+len(rest)] * t[T] for T in range(len(TPR))) + xsum(A[r][Z+len(flights)+len(rest)+len(TPR)] * z[Z] for Z in range(len(ZQ))) <= b[r]    

            # OPTIMIZATION #

            m.optimize()
            m.objective_value

            #################################################################################################################################################

            # Objective Function Value #

            m.objective_value
            #print('\n................\n')
            #print('Objective Function: "{}" '.format(m.objective_value))
            #print('\n................\n')

            #################################################################################################################################################

            # Displaying The Output of Fleet Assignment #

            Dec_Var = [i.x for i in m.vars]


            Flight_ans = []
            Fleet_ans = []

            index = 0
            for (j, i) in product(range(F),range(K)):
                if Dec_Var[index] == 1.0: 
                    #print("Flight Number %d is Assigned by Fleet Type %d \n" % (j+1, i+1))
                    Flight_ans.append(j+1)
                    Fleet_ans.append(i+1)

                index = index + 1


            Passengers = []
            ti = []
            tj = []

            index_tpr = len(flights) + len(rest)
            for (j, i) in product(range(I),range(I+1)):
                if Dec_Var[index_tpr] != 0.0: 
                    #print(" %d Passengers are Recaptured from Itinerary %d by Itinerary %d \n" % (Dec_Var[index_tpr], j+1, i+1))
                    Passengers.append(Dec_Var[index_tpr])
                    ti.append(j+1)
                    tj.append(i+1)

                index_tpr = index_tpr + 1

            print(Passengers)
            print(ti)
            print(tj)


        #return({'cost':cost})
        print({'objective_value': m.objective_value, 'Flight_ans':Flight_ans, 'Fleet_ans':Fleet_ans ,'Passengers':Passengers ,'ti':ti ,'tj':tj})
        return({'objective_value': m.objective_value, 'Flight_ans':Flight_ans, 'Fleet_ans':Fleet_ans , 'Passengers':Passengers ,'ti':ti ,'tj':tj })

    
api.add_resource(FleetAssignment, '/FleetAssignment')
api.add_resource(Solution, '/Solution')


   
if __name__ == '__main__':
    app.run()

#######################################################################################################################

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jun/2022 13:42:13] "GET /FleetAssignment HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2022 13:42:40] "GET /FleetAssignment HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2022 13:43:33] "OPTIONS /FleetAssignment HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2022 13:43:33] "POST /FleetAssignment HTTP/1.1" 200 -


{'i_orgin': 'Anaa Airport', 'i_destination': 'El Mellah Airport', 'I_Opt_Mand': 'mandatory', 'no_connecting_flights': 1, 'demand': 50, 'fare': 200, 'K': 2, 'OC': [30000, 16600], 'cap': [150, 250], 'iti': [1], 'size_K': [3, 3]}
[30000, 16600] [30000, 16600]
[3, 3]


127.0.0.1 - - [19/Jun/2022 13:44:16] "OPTIONS /FleetAssignment HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2022 13:44:16] "POST /FleetAssignment HTTP/1.1" 200 -


{'i_orgin': 'El Mellah Airport', 'i_destination': 'Aalborg Airport', 'I_Opt_Mand': 'optional', 'no_connecting_flights': 1, 'demand': 50, 'fare': 225, 'K': 2, 'OC': [15100, 13500], 'cap': [150, 250], 'iti': [2], 'size_K': [3, 3]}
[15100, 13500] [30000, 16600, 15100, 13500]
[3, 3]


...

{'i_orgin': 'Anaa Airport', 'i_destination': 'Aalborg Airport', 'I_Opt_Mand': 'mandatory', 'no_connecting_flights': 1, 'demand': 300, 'fare': 350, 'K': 2, 'OC': [15500, 45000], 'cap': [150, 250], 'iti': [3], 'size_K': [3, 3]}
[15500, 45000] [30000, 16600, 15100, 13500, 15500, 45000]
[3, 3]


127.0.0.1 - - [19/Jun/2022 13:45:01] "OPTIONS /Solution HTTP/1.1" 200 -


{'solvess': 'solve'}


127.0.0.1 - - [19/Jun/2022 13:45:02] "POST /Solution HTTP/1.1" 200 -


[None, None, None, None, None, None, None, None, None, None, None, None]
[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
{'objective_value': None, 'Flight_ans': [], 'Fleet_ans': [], 'Passengers': [None, None, None, None, None, None, None, None, None, None, None, None], 'ti': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3], 'tj': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]}
